# Simulation study
### Setting:
- $\beta = 0.003$
- $5000$ miners
- No measurement error correction (using true data, i.e. X not Z)

Written to `b3-5-true`

In [7]:
# general libraries
import os
import numpy as np
import scipy.stats as stats
import warnings
import multiprocessing
import time
import matplotlib.pyplot as plt
import seaborn as sns


# sampling code
import sys
sys.path.append('..')
import wismut.basics as basics
from wismut.MCMC import MCMC
import wismut.analyze_chains as ac
path = os.getcwd() + "/"

### Define prior parameters

In [8]:
def generate_prior_parameters():
    prior_parameters = {'beta': {'dist': "normal", 'mean': 0, 'sd': 200},
                        'lambda1': {'dist': "gamma",'shape': 600,
                                    'scale': 1 / 10000000,
                                    'min': 0, 'max': 200
                                    },
                        'lambda2': {'dist': "gamma", 'shape': 12000,
                                    'scale': 1 / 1000000,
                                    'min': 0, 'max': 200
                                    },
                        'lambda3': {'dist': "gamma", 'shape': 46000,
                                    'scale': 1 / 1000000,
                                    'min': 0, 'max': 200
                                    },
                        'lambda4': {'dist': "gamma", 'shape': 1000,
                                    'scale': 1 / 100000,
                                    'min': 0, 'max': 200
                                    },
                        }
    return prior_parameters



### Deinfe Proposal sds for MCMC

In [9]:
def generate_proposal_sds(disease_model='cox'):
    proposal_sd = {
            'beta': 0.011 if disease_model == 'cox' else 0.00011*10,
            'lambda1': 0.000211,
            'lambda2': 0.000611,
            'lambda3': 0.000611,
            'lambda4': 0.000211,
            }
    return proposal_sd

### Define Start values of Markov chains

In [10]:
def generate_start_values(seed, chain, disease_model="cox_like", me_correction=True):
    np.random.seed(seed)
    rnd = lambda: stats.uniform(loc=0.9, scale=0.2).rvs(1)[0]
    
    beta_true = 0.3 if disease_model == "cox_like" else 1.0
    l1 = 0.00006
    l2 = 0.00120
    l3 = 0.00460
    l4 = 0.01000

    start_values = {chain: {'beta': beta_true * rnd(),
                               'lambda1': l1 * rnd(),
                               'lambda2': l2 * rnd(),
                               'lambda3': l3 * rnd(),
                               'lambda4': l4 * rnd(),
                               }
                    }
    if not me_correction:
        del start_values[chain]['prior_parameters']

    return start_values


### Define uncertainty characteristics

In [11]:
###########################
# M1a M2 M2_Expert M3 M4
###########################
uncertainty_characteristics = {
        'M1a': {'C_Rn_old': {'classical_error': {'sd': 0, 'structure': 'additive', 'proposal_sd': 0.1},
                             'Berkson_error': {'sd': 0},
                             'name_obs_values': 'C_Rn_old_true'
                             },
                'C_Rn_ref': {'classical_error': {'sd': 0, 'structure': 'additive', 'proposal_sd': 0.1},
                             'Berkson_error': {'sd': 0},
                             'name_obs_values': 'C_Rn_ref_true'
                             },
                'b': {'classical_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.1},
                      'Berkson_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.1},
                      'name_obs_values': 'b_Berkson'
                      },
                'tau_e': {'classical_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.1},
                          'Berkson_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.1},
                          'name_obs_values': 'tau_e_Berkson'
                          },
                'A': {'classical_error': {'sd': 0},
                      'Berkson_error': {'sd': 0},
                      'name_obs_values': 'A_calculated'
                      },
                'A_ref': {'classical_error': {'sd': 0},
                          'Berkson_error': {'sd': 0},
                          'name_obs_values': 'A_ref'
                          },
                'r': {'classical_error': {'sd': 0},
                      'Berkson_error': {'sd': 0},
                      'name_obs_values': 'r'
                      },
                },
        'M2': {'C_Rn': {'classical_error': {'sd': 0, 'structure': 'additive', 'proposal_sd': 0.1},
                        'Berkson_error': {'sd': 0},
                        'name_obs_values': 'C_Rn_true'
                        },
               },
        'M2_Expert': {'C_Exp': {'classical_error': {'sd': 0, 'structure': 'additive', 'proposal_sd': 0.1},
                                'Berkson_error': {'sd': 0},
                                'name_obs_values': 'C_Rn_true',
                                },
                   },
        'M3': {'C_RPD': {'classical_error': {'sd': 0, 'structure': 'additive', 'proposal_sd': 0.1},
                         'Berkson_error': {'sd': 0},
                         'name_obs_values': 'C_Rn_true'
                         },
               'zeta': {'classical_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.1},
                        'Berkson_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.1},
                        'name_obs_values': 'c_Berkson'
                        },
               },
        'M4': {'E_Rn': {'classical_error': {'sd': 0, 'structure': 'additive', 'proposal_sd': 0.1},
                        'Berkson_error': {'sd': 0},
                        'name_obs_values': 'C_Rn_true'
                        },
               },
    
        'activity': {'phi': {'classical_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.1},
                             'Berkson_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.1},
                             'name_obs_values': 'f_Berkson'
                             },

                     },
        'working_time': {'omega': {'classical_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.1},
                                   'Berkson_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.1},
                                   'name_obs_values': 'w_Berkson'
                                   }
                         },
        'equilibrium': {'gamma': {'classical_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.01},
                                  'Berkson_error': {'sd': 0, 'structure': 'multiplicative', 'proposal_sd': 0.1},
                                  'name_obs_values': 'g_Berkson'
                                  },
                        },
        }

# # # #  only one chain
data = basics.read_data(path + f"../data/M1M2M2_ExpertM3M4-b3-5/Data_1.csv")

data['tau'] = 1

disease_model = "cox_like"
chain = 'chain1'
seed = 123
start_values = generate_start_values(seed=seed, chain=chain, disease_model=disease_model)
proposal_sd = generate_proposal_sds()
prior_parameters = generate_prior_parameters()

s = np.array([0, 40, 55, 75, 104])
path_results = path + '../results/simulation_study/b3-5-true/'
mcmc = MCMC(data=data, uncertainty_characteristics=uncertainty_characteristics,
            s=s, path_results=path_results, proposal_sd=proposal_sd,
            prior_parameters=prior_parameters, start_values=start_values,
            informative_priors=False, chain=chain,
            disease_model=disease_model, fixed_parameters=False)

iterations = 1000; burnin = 1; phases = 1; thin = 1
mcmc.run_adaptive_algorithm(iterations=iterations, burnin=burnin,
                            adaptive_phases=phases, save_chains=True,
                            thin=thin)

x = np.genfromtxt(path_results + 'chain1_results_beta.txt', delimiter=',')
plt.plot(x); plt.show()

### Define function to run one chain on one dataset

In [12]:
def run_dataset(nb, seed, chain):
    data = basics.read_data(path + f"../data/M1M2M2_ExpertM3M4-b3-5/Data_{nb}.csv")
    data['tau'] = 1
    path_results = path + f'../results/simulation_study/b3-5-true/{nb}/'
    # print(path_results)
    os.makedirs(path_results, exist_ok=True)

    disease_model = 'cox_like'

    start_values = generate_start_values(seed, chain, disease_model)
    proposal_sd = generate_proposal_sds()
    prior_parameters = generate_prior_parameters()
    s = np.array([0, 40, 55, 75, 104])

    mcmc = MCMC(data=data,uncertainty_characteristics=uncertainty_characteristics, s=s, path_results=path_results, proposal_sd=proposal_sd,
            prior_parameters=prior_parameters, start_values=start_values,
            informative_priors=False, chain=chain, 
            disease_model=disease_model, fixed_parameters=False)


    iterations = 100_000; burnin = 20_000; phases = 100; thin = 100
    # iterations = 10_000; burnin = 2_000; phases = 10; thin = 10
    mcmc.run_adaptive_algorithm(iterations=iterations,
                                burnin=burnin,
                                adaptive_phases=phases,
                                thin=thin,
                                clear_display=True
                               )

### Run 4 chains on 100 datasets

In [ ]:
nb_data_sets = 100
cores = 140
cores = 35

datasets = list(range(1,nb_data_sets+1))*4
seeds = np.random.randint(2**32-1, size=nb_data_sets*4)
chains = [*["chain1"]*nb_data_sets, *["chain2"]*nb_data_sets,  *["chain3"]*nb_data_sets,  *["chain4"]*nb_data_sets]
arg_list = list(zip(datasets, seeds, chains))
warnings.filterwarnings('ignore')

t = time.time()
with multiprocessing.Pool(cores) as pool:
    res = pool.starmap(run_dataset, arg_list)
t = time.time() - t
print("Full calculation time: " + str(time.time() - t))
print(f"Time in hours: {t/3600}")

Adaptive phase 6


In [14]:
print("Full calculation time: " + str(time.time() - t))
print(f"Time in hours: {t/3600}")
# Full calculation time: 1716337568.6616385
# Time in hours: 12.191403737531768

Full calculation time: 1740930620.9152997
Time in hours: 17.659178157978587


## Load results and make diagnostics

In [ ]:
samples = ac.load_traces_parameter("beta.txt", "../results/simulation_study/b3-5-true/")

In [ ]:
ac.plot_chains_sim(samples, (1,20))

In [ ]:
# Check results
res_reduced = ac.reduce_results(samples, (1,101))
inside = [ac.check_inside(0.3, res_reduced[k]["hdi"]) for k in range(1,101)]
means = [res_reduced[k]["mean"] for k in range(1,101)]

print(f"Overalll mean: {np.array(means).mean()}")
print(f"Valids: {np.array(inside).mean()}")
print(f"Rhats (sorted): {np.sort(np.concatenate([x.to_array().to_numpy() for x in ac.calc_rhat(samples, (1,101), simplify=True)]))}")

### Lambda

In [ ]:
l1 = ac.load_traces_parameter("lambda1.txt", "../results/simulation_study/b3-5-naive/")
l2 = ac.load_traces_parameter("lambda2.txt", "../results/simulation_study/b3-5-naive/")
l3 = ac.load_traces_parameter("lambda3.txt", "../results/simulation_study/b3-5-naive/")
l4 = ac.load_traces_parameter("lambda4.txt", "../results/simulation_study/b3-5-naive/")

print(f"Rhats lambda1: {np.concatenate([x.to_array().to_numpy() for x in ac.calc_rhat(l1, (1,101), simplify=True)])}")
print(f"Rhats lambda2: {np.concatenate([x.to_array().to_numpy() for x in ac.calc_rhat(l2, (1,101), simplify=True)])}")
print(f"Rhats lambda3: {np.concatenate([x.to_array().to_numpy() for x in ac.calc_rhat(l3, (1,101), simplify=True)])}")
print(f"Rhats lambda4: {np.concatenate([x.to_array().to_numpy() for x in ac.calc_rhat(l4, (1,101), simplify=True)])}")

In [ ]:
l1_reduced = ac.reduce_results(l1, (1,101))
l2_reduced = ac.reduce_results(l2, (1,101))
l3_reduced = ac.reduce_results(l3, (1,101))
l4_reduced = ac.reduce_results(l4, (1,101))

l1_means = [l1_reduced[k]["mean"] for k in range(1,101)]
l2_means = [l2_reduced[k]["mean"] for k in range(1,101)]
l3_means = [l3_reduced[k]["mean"] for k in range(1,101)]
l4_means = [l4_reduced[k]["mean"] for k in range(1,101)]

print(f"Overalll mean: {np.array(l1_means).mean()}")
print(f"Overalll mean: {np.array(l2_means).mean()}")
print(f"Overalll mean: {np.array(l3_means).mean()}")
print(f"Overalll mean: {np.array(l4_means).mean()}")